In [ ]:
#default_exp encrypt

# encrypt
> need pycryptodome
Encrypt data with various methods eg AES

In [4]:
#hide
from nbdev.showdoc import *

# AES

In [5]:
#export
from Crypto.Cipher import AES
from beartype import beartype
from typing import Tuple


## Encrypt

In [6]:
#export
@beartype
def encryptAes(key:bytes, data:bytes)-> Tuple[bytes,bytes]:
  '''
    note that key has to be 16 bits long
    response:
      cipherText in bytes
  '''
  cipher = AES.new(key, AES.MODE_EAX)
  ciphertext, tag = cipher.encrypt_and_digest(data)
  return ciphertext, cipher.nonce


In [10]:
key = b'testtesttesttest'
data = b'test'
ciphertext, nonce = encryptAes(key,data)
print(ciphertext, nonce)

b'z\xac\x0f0' b'\x03?\x00\xd6\x89\x86\xf2\x1d<\xc0 d\xd6h.\xc2'


## Decrypt

In [11]:
#export
@beartype
def decryptAes(key:bytes, ciphertext:bytes, nonce:bytes)->bytes:
  cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
  plaintext = cipher.decrypt(ciphertext)
  return plaintext

In [12]:
decryptAes(key, ciphertext, nonce)

b'test'